</div>
<div align="center">
  <img src="img/evolnode.png" width="800" alt="Fourier reconstruction convergence">
  <p><em> Evolution Node free you from coding and debugging, let LLM evolve your code for you.</em></p>
</div>
<div align="left">
<p><em>Bored of manual coding, a function? EvolNode let LLM automate function design and guide the evolution of it with genetic algorithm. A node here takes a task, input, and output, it uses either code or another LLM to complete the task, ensuring aligned input and output value types and ,.names.Fitness evaluation is done by running the function with a few specified test cases, the more diverse the test case, the better the evolution.
</em></p>
</div>

In [1]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode
from methods.llm import get_groq_response

# Code + Compilor Task
mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)

test_cases = [
    {"name": "Dilireba", "age": 32},
    {"name": "ChengXiao", "age": 26},
]

test_inputs = [{"name": "Dilireba"}, {"name": "ChengXiao"}]

node = EvolNode(mp, None, None, get_response=get_groq_response, test_cases=test_cases)



# reasoning, code = node.evolve("i1", replace=True)
# input_dict = {"n": 10}
# output_dict = node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
# mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 
# node = EvolNode(mp, None, None, get_response=get_groq_response)
# node.evolve("i1", replace=True)

# input_dict = {"name": "Dilireba"}
# output_dict = node(input_dict) 

Could not load vllm class, check CUDA support and GPU RAM size


In [7]:
# node.evolve("i1", replace=True, num_runs=1) # Wrong error message, STH funny in here

self = node
method = "i1"
parents = None
replace = False
num_runs = 1


reasoning, code = self._evolve(method, parents, replace=False)    
# _, fitness, error_msg = self._evaluate_fitness(code=code, max_tries=1, num_runs=num_runs)     


In [9]:
import datetime
import requests
from bs4 import BeautifulSoup
def get_celeb_age(name: str) -> int:
    """
    This function calculates the current age of a celebrity given their name.
    
    Parameters:
    name (str): The name of the celebrity.

    Returns:
    int: The current age of the celebrity.
    """
    url = 'https://www.google.com/search?q=' + '+'.join(name.split())
    response = requests.get(url)
    if response.status_code == 200:
        page_content = response.content
        soup = BeautifulSoup(page_content, 'html.parser')
        tables = soup.find_all('table')
        table = tables[0]
        rows = table.find_all('tr')
        birthday = ''
        for row in rows:
            cols = row.find_all('td')
            if len(cols) >= 2:
                col1, col2 = (cols[0].text.strip(), cols[1].text.strip())
                if 'Born' in col1 or 'Birth' in col1:
                    birthday = col2
        if not birthday:
            return 0
        birthday_year = int(birthday.split(',')[1].strip().split()[0])
        current_year = datetime.date.today().year
        age = current_year - birthday_year
        return age
    else:
        return 0
    
get_celeb_age("Dilireba")

KeyboardInterrupt: 

In [33]:
# Build a query-google node 

from methods.meta_prompt import MetaPrompt
from methods.eoh_evolution import EvolNode 

mp = MetaPrompt("Query Google for result", "query_google", ["query"], ["result"], ["str"], ["str"], PromptMode.CODE)

code_string = '''
import http.client
import json
import os
from typing import Dict, Any

def _search_google(query: str) -> Dict[str, Any]:
    """
    Use Serper API to query Google for information
    
    Args:
        query (str): The search query
    
    Returns:
        Dict[str, Any]: Parsed JSON response from the API
    """
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ["SERPER_API_KEY"],
        'Content-Type': 'application/json'
    }
    
    try:
        conn.request("POST", "/search", payload, headers)
        res = conn.getresponse()
        data = res.read()
        return json.loads(data.decode("utf-8"))
    except Exception as e:
        print(f"Error occurred during API request: {str(e)}")
        return {}
    finally:
        conn.close()
        
def search_google(query: str) -> str: 
    """ 
    Input query, return search result string from Google
    """
    result = _search_google(query)
    result_dict = {k.replace("organic", "Search Result"): v for k, v in result.items() if k in ["answerBox", "organic"]}
    result_str = json.dumps(result_dict, indent=2)
    return result_str 
'''


node = EvolNode(mp, code=code_string, reasoning="Query google for top search results", get_response=get_groq_response)

# node.evolve("i1", replace=True)



Generating test cases: 5case [00:00,  7.05case/s]             

--- Generated 5 test cases


In [36]:
node.save()

In [34]:
node.test_cases

[({'query': 'Python programming'},
  {'result': '...example search results for Python programming...'}),
 ({'query': ''}, {'result': 'Error: Query cannot be empty'}),
 ({'query': None}, {'result': 'Error: Query cannot be null'}),
 ({'query': ' abc def'},
  {'result': '...resultados de búsqueda para abc def...'}),
 ({'query': '  Multiple  Space  Query  '},
  {'result': '...example search results for Multiple Space Query...'})]

In [31]:
import http.client
import json
import os
from typing import Dict, Any

def _search_google(query: str) -> Dict[str, Any]:
    """
    Use Serper API to query Google for information
    
    Args:
        query (str): The search query
    
    Returns:
        Dict[str, Any]: Parsed JSON response from the API
    """
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ["SERPER_API_KEY"],
        'Content-Type': 'application/json'
    }
    
    try:
        conn.request("POST", "/search", payload, headers)
        res = conn.getresponse()
        data = res.read()
        return json.loads(data.decode("utf-8"))
    except Exception as e:
        print(f"Error occurred during API request: {str(e)}")
        return {}
    finally:
        conn.close()
        
def search_google(query: str) -> str: 
    """ 
    Input query, return search result string from Google
    """
    result = _search_google(query)
    result_dict = {k.replace("organic", "Search Result"): v for k, v in result.items() if k in ["answerBox", "organic"]}
    result_str = json.dumps(result_dict, indent=2)
    return result_str 


search_google("Dilireba")

'{\n  "Search Result": [\n    {\n      "title": "Dilraba Dilmurat - Wikipedia",\n      "link": "https://en.wikipedia.org/wiki/Dilraba_Dilmurat",\n      "snippet": "Dilraba Dilmurat (Uyghur: \\u062f\\u0649\\u0644\\u0631\\u06d5\\u0628\\u0627 \\u062f\\u0649\\u0644\\u0645\\u06c7\\u0631\\u0627\\u062a, Chinese: \\u8fea\\u4e3d\\u70ed\\u5df4\\u00b7\\u8fea\\u529b\\u6728\\u62c9\\u63d0; born June 3, 1992) is a Chinese actress, singer and model. She is an ethnic Uyghur ...",\n      "position": 1\n    },\n    {\n      "title": "Dilraba Dilmurat (\\u8fea\\u4e3d\\u70ed\\u5df4) - MyDramaList",\n      "link": "https://mydramalist.com/people/7830-dilmurat-dilraba",\n      "snippet": "Dilraba Dilmurat, born in Xinjiang, China, is an award-winning actor, dancer and singer of Uyghur descent.",\n      "sitelinks": [\n        {\n          "title": "Love Beyond the Grave",\n          "link": "https://mydramalist.com/750689-bai-ri-ti-deng"\n        },\n        {\n          "title": "Love on the Turquoise Land"

In [23]:
# result_dict = json.dumps(result, indent=2)
type(result)

dict

In [28]:
{k.replace("organic", "Search Result"): v for k, v in result.items() if k in ["answerBox", "organic"]}

{'answerBox': {'title': 'Dilraba Dilmurat / Age', 'answer': '32 years'},
 'Search Result': [{'title': 'Dilraba Dilmurat - Wikipedia',
   'link': 'https://en.wikipedia.org/wiki/Dilraba_Dilmurat',
   'snippet': '(1992-06-03) June 3, 1992 (age 32). Ürümqi, Xinjiang, China. Alma mater ... Dilireba Studio. Uyghur name. Uyghur, دىلرەبا دىلمۇرات\u200e. show. Transcriptions.',
   'position': 1},
  {'title': 'Dilraba Dilmurat (迪丽热巴) - MyDramaList',
   'link': 'https://mydramalist.com/people/7830-dilmurat-dilraba',
   'snippet': 'Dilireba, Dili Reba, Dilireba Dilimulati, Di Li Re Ba, Pan Di, Re Ba, Xiao ... Age: 32. Dilraba Dilmurat, born in Xinjiang, China, is an award-winning ...',
   'position': 2},
  {'title': 'Dilraba Dilmurat - Simple English Wikipedia, the free encyclopedia',
   'link': 'https://simple.wikipedia.org/wiki/Dilraba_Dilmurat',
   'snippet': '... Dílìrèbā Dílìmùlātí, born June 3, 1992) is ... (1992-06-03) June 3, 1992 (age 32). Ürümqi, Xinjiang, China. Nationality, Chinese. Ot

In [12]:
res = search_google("Dilireba")
type(res)

str

In [16]:
import json
data = json.loads(res)


In [18]:
data["knowledgeGraph"]

{'title': 'Dilraba Dilmurat',
 'type': 'Chinese actress and singer',
 'imageUrl': 'https://encrypted-tbn0.gstatic.com/licensed-image?q=tbn:ANd9GcSLaws8RuT-oGUOOD0ooCTBPd9-FK_3Av2NgsqKqVXYldMr_htJlMGKbQCFW0G-VYVEhANP&s=19',
 'description': 'Dilraba Dilmurat is a Chinese actress, singer and model. She is an ethnic Uyghur from Ürümqi, Xinjiang.',
 'descriptionSource': 'Wikipedia',
 'descriptionLink': 'https://en.wikipedia.org/wiki/Dilraba_Dilmurat',
 'attributes': {'Born': '1992 (age 32\xa0years), Ürümqi, China',
  'Height': '5′ 7″',
  'Nationality': 'Chinese',
  'Traditional Chinese': '迪麗熱巴·迪力木拉提',
  'Uyghur': 'دىلرەبا دىلمۇرات\u200e'}}

In [5]:
node.call_code_func(test_inputs[1], node.code)

PageError: Page id "cheng xiao" does not match any pages. Try another id!

In [20]:

ERROR_ANALYSIS_PROMPT = """Analyze the error message, suggest where the issue is and how to fix it. Be concise.
Error Message:
{error_message}
"""

error_analysis_prompt = ERROR_ANALYSIS_PROMPT.format(error_message=errors[0])

response = get_groq_response(error_analysis_prompt)


In [21]:
print(response)

**Error Analysis:**

The error message indicates that there is an `IndexError` with the error type "list index out of range". This occurs when a Python list is accessed with an index that is out of the list's bounds.

**Issue Location:**

The issue is likely at the following part of the code:
`result = node.call_code_func(test_inputs[0], node.code)`

Here, `test_inputs[0]` is being accessed as if it's a list, but it seems that it's not long enough to have at least one element.

**Fix:**

1. Ensure that `test_inputs` is a list and has at least one element.
2. If `test_inputs` is supposed to be a single value, consider removing the indexing (`test_inputs` without `[0]`).
3. Check the length of `test_inputs` before attempting to access `test_inputs[0]`. Add a conditional statement to handle the case when `test_inputs` is empty or if `test_inputs[0]` doesn't exist:

```python
if len(test_inputs) > 0:
    result = node.call_code_func(test_inputs[0], node.code)
else:
    # Handle the case wh

In [26]:
import wikipedia
from datetime import date
def get_celeb_age(name: str) -> int:
    """
    Calculate the age of a celebrity based on their Wikipedia page.

    Args:
        name (str): The celebrity's name.

    Returns:
        int: The celebrity's current age.
    """
    try:
        page = wikipedia.page(name)
    except wikipedia.exceptions.DisambiguationError:
        raise Exception(f'Multiple pages found for {name}')
    except wikipedia.exceptions.PageError:
        raise Exception(f'No page found for {name}')
    wiki_text = page.content
    birth_year = None
    for line in wiki_text.splitlines():
        if 'born' in line.lower():
            words = line.split()
            for word in words:
                if word.isdigit():
                    birth_year = int(word)
                    break
            break
    if birth_year is None:
        raise Exception(f'Birth year of {name} not found')
    current_year = date.today().year
    age = current_year - birth_year
    return age

get_celeb_age("Dilireba")

Exception: Birth year of Dilireba not found

In [17]:
from datetime import date
import csv
from bs4 import BeautifulSoup
import requests
def get_celeb_age(name: str) -> int:
    """
    Returns the current age of a celebrity.

    Args:
    name (str): The celebrity's name.

    Returns:
    int: The current age of the celebrity.
    """
    url = f"https://en.wikipedia.org/wiki/{name.replace(' ', '_')}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    birth_date_tag = soup.find('span', class_='bday')
    if birth_date_tag is None:
        birth_date_tag = soup.find('span', class_='birth-date')
    if birth_date_tag is not None:
        birth_date_str = birth_date_tag.text.strip()
        birth_date_str = birth_date_str.replace('born', '').strip()
        birth_day, birth_month, birth_year = birth_date_str.split('-')
        birth_year = int(birth_year)
        birth_month = int(birth_month)
        birth_day = int(birth_day)
    else:
        return None
    today = date.today()
    age = today.year - birth_year - ((today.month, today.day) < (birth_month, birth_day))
    return age

2021

In [14]:
birth_date_tag.text.strip()

'1992-06-03'

In [2]:
# response = node._get_evolve_response("i1")

# node._evolve("i1")

# node.evolve("i1", replace=True, num_runs=1)

# node.test_inputs
node.test_cases[0]

input_dict = {"name": "Dilireba"}
err_msg = ""
try: 
    node.call_code_func(input_dict, node.code)
except Exception as e:
    err_msg += str(e)


In [4]:
print(node.code)

None


In [10]:
node.meta_prompt

MetaPrompt(task='Search for age of a celebrity.', func_name='get_celeb_age', inputs=['name'], outputs=['age'], input_types=['str'], output_types=['int'], mode=<PromptMode.CODE: 'code'>)

#### Direct Loading Node

In [1]:
from methods.eoh_evolution import EvolNode 

node = EvolNode.load("get_celeb_age")
input_dict = {"name": "Dilireba"}
node(input_dict)

Could not load vllm class, check CUDA support and GPU RAM size


{'age': 31}

#### Alignment-Check for Evaluation

In [2]:
from methods.llm import get_groq_response 

# Evaluation base on alignment check
node.get_response = get_groq_response 
structure_fitness, function_fitness, issue_summary = node._evaluate_fitness(max_tries=1, num_runs=2)

Evaluating fitness: 100%|██████████| 4/4 [00:13<00:00,  3.39s/it]


In [3]:
print(issue_summary)

--- Compiled 0 out of 4 test cases
--- Passed 0 out of 4 test cases
Given Input: {'name': 'Brad Pitt'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Jennifer Lawrence'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Brad Pitt'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Jennifer Lawrence'}. Can't parse output dictionary from LLM's response.



In [5]:
# how to put a node into another nodes? 
# Ideally I would then ask for the node (which likely fails on many of the test-cases) to come-up with a code, where it could call on some 'pseudo-node'
# These pseudo-nodes are used, very similar to a pseudo-code 




</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>

</div>
<div align="center">
  <img src="img/Planning-node.png" width="400" alt="Planning Node Task Decomposition">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [1]:
from methods.llm import get_claude_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan,extract_python_code, extract_json_from_text

# Step 0. Initialize MetaPlanning Prompt
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])

# Step 1: Generate Pseudo-Code for SubTask Decomposition
prompt = mp._get_pseudo_code_prompt() # Pseudo-Code Prompt (Non-implemented functional)
response = get_claude_response(prompt) # Use Strong LLM to build up pseudo-code
code = extract_python_code(response) # Extract Python Code from response 

# Step 2: Generate Planning DAG: Multiple Nodes 
graph_prompt = mp._get_plan_graph_prompt(code) 
plan_response = get_claude_response(graph_prompt)
plan_dict = extract_json_from_text(plan_response)

visualize_plan_dict(plan_dict)

# Step 3: Spawn Multiple Sub-Nodes and Evolve them

# Step 4: Slot Sub Node into context for parent node re-write 

Could not load vllm class, check CUDA support and GPU RAM size


success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 207.111084ms
success: successfully compiled d2_output/plan_graph_dag.d2 to d2_output/plan_graph_dag.png in 207.980416ms


In [1]:
# Ok we could load this thing now ..

from methods.eoh_evolution import PlanNode 
from methods.llm import get_groq_response 

plan_node = PlanNode.load("get_celeb_age", get_response=get_groq_response)

Could not load vllm class, check CUDA support and GPU RAM size


In [5]:
plan_dict = plan_node.plan_dict

In [6]:
prompt_nodes, code_nodes = plan_node._spawn_nodes(plan_dict)

Generating test cases: 5case [00:05,  1.20s/case]             


--- Generated 5 test cases


Generating test cases: 5case [00:09,  1.99s/case]             


--- Generated 5 test cases


Generating test cases: 5case [00:06,  1.20s/case]             


...

--- Generated 3 test cases


Generating test cases: 5case [00:06,  1.28s/case]             


--- Generated 5 test cases


Generating test cases: 5case [00:10,  2.19s/case]             

--- Generated 5 test cases


In [17]:
# Chicken n Egg problem: - how can you evolve a function which you can't evaluate? 
code_nodes["scrape_celeb_data"].test_cases # probably not fair to expect test-cases for this one to be grounded ... 

[({'name': 'Tom Hanks'}, {'data': '67'}),
 ({'name': 'Emma Watson'}, {'data': '33'}),
 ({'name': 'Leonardo da Vinci'}, {'data': 'Deceased'})]

In [18]:
scrape_node = code_nodes["scrape_celeb_data"]

scrape_node.evolve("i1", replace=True)

Evaluating fitness: 100%|██████████| 15/15 [00:36<00:00,  2.43s/it]


 - Attempt 1 failed. Fitness: 0.00. Error: --- Compiled 10 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Input: {'name': 'Emma Watson'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': '33'}
Input: {'name': 'Leonardo da Vinci'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': 'Deceased'}
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Input: {'name': 'Emma Watson'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': '33'}
Input: {'name': 'Leonardo da Vinci'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': 'Deceased'}
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Input: {'name': 'Emma Watson'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': '33'}
Input: {'name': 'Leonardo da Vinci'}, Pred: {'data': 'Celebrity not found.'}, Expected: {'data': 'Deceased'}
Given I

Evaluating fitness: 100%|██████████| 15/15 [00:00<00:00, 44906.90it/s]


 - Attempt 2 failed. Fitness: 0.00. Error: --- Compiled 0 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM

Evaluating fitness: 100%|██████████| 15/15 [00:00<00:00, 31583.61it/s]


 - Attempt 3 failed. Fitness: 0.00. Error: --- Compiled 0 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM

Evaluating fitness: 100%|██████████| 15/15 [00:00<00:00, 7772.03it/s]


 - Attempt 4 failed. Fitness: 0.00. Error: --- Compiled 0 out of 15 test cases
--- Passed 0 out of 15 test cases
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Emma Watson'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Leonardo da Vinci'}. Can't parse output dictionary from LLM's response.
Given Input: {'name': 'Tom Hanks'}. Can't parse output dictionary from LLM

Evaluating fitness:   0%|          | 0/15 [01:21<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Code Node evolution is bad now, parsing seems to be problematic



In [20]:
code = scrape_node.code
print(code)

defined structured data to locate the celebrity based on their name and then retrieving their corresponding age.}

```python
def scrape_celeb_data(name: str) -> str:
    # Predefined mock data to simulate web data
    celeb_data = {
        "Taylor Swift": 33,
        "Leonardo DiCaprio": 48,
        "Beyoncé": 42,
        "Chris Evans": 42,
        "Scarlett Johansson": 38
    }
    
    # Attempt to find the celebrity's age
    age = celeb_data.get(name)

    # If age not found, return a message
    if age is None:
        return f"Age for {name} not found."
    
    # Return the age as a string
    return f"{name} is {age} years old."


In [10]:
# Evolve Sub-Nodes (hope is not too slow here ...)
# 1. Code success -> end | else -> Prompt | pick best -> return node for specific sub-task

# prompt_nodes, code_nodes = plan_node._spawn_nodes(plan_dict)

In [24]:
# visualize_plan_dict(plan_dict)

plan_dict # Ignore the mode for now, let's use both mode and pick the best performing ones ... 

plan_dict["nodes"]

# Basic Idea: 
# - Build node libraries is the better way to describe this 



[{'task': 'Search for celebrity in database',
  'name': 'search_celebrity_database',
  'inputs': ['name'],
  'input_types': ['str'],
  'outputs': ['celeb_info'],
  'output_types': ['dict'],
  'target': 'Retrieve celebrity information',
  'mode': 'CODE'},
 {'task': 'Extract birth date from celebrity info',
  'name': 'extract_birth_date',
  'inputs': ['celeb_info'],
  'input_types': ['dict'],
  'outputs': ['birth_date'],
  'output_types': ['date'],
  'target': "Get celebrity's birth date",
  'mode': 'CODE'},
 {'task': 'Get current date',
  'name': 'get_current_date',
  'inputs': [],
  'input_types': [],
  'outputs': ['current_date'],
  'output_types': ['date'],
  'target': 'Obtain current date for age calculation',
  'mode': 'CODE'},
 {'task': 'Calculate age based on birth date and current date',
  'name': 'calculate_age',
  'inputs': ['birth_date', 'current_date'],
  'input_types': ['date', 'date'],
  'outputs': ['age'],
  'output_types': ['int'],
  'target': "Determine celebrity's curr